#IST718 Project - Google and NCAA Women's Basketball Tournament Prediction

#Data Exploration and Descriptive Statistics

@authors
Sanjana Rajagopala,
Shefali Vajramatti,
Apoorva Rajendra Angre,
Sandya Madhavan

In [2]:
#IMPORT ALL THE REQUIRED PACKAGES
import pandas as pd
from pyspark.ml import feature
from pyspark.ml.classification import LogisticRegression
#from Regr import LinearRegression
from pyspark.ml import classification
from pyspark.ml import Pipeline
from pyspark.sql import SQLContext
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import functions as fn
from pyspark.sql.types import IntegerType
import numpy as np
from pyspark.mllib.evaluation import BinaryClassificationMetrics as metric
from pyspark.sql import functions as sf
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier



import matplotlib.pyplot as plt

In [3]:
#Read the required data from the CSV files uploaded in the FileStore dbfs of the Databricks environment
wteamDF = spark.read.csv("/FileStore/tables/WTeams.csv", header=True, inferSchema= True)
lteamDF = spark.read.csv("/FileStore/tables/WTeams.csv", header=True, inferSchema= True)

#Read the RegularSeasons CSV File
regularSeasonsDF = spark.read.csv("/FileStore/tables/WRegularSeasonCompactResults.csv", header=True, inferSchema= True)

#Read the Seeds and Slots CSV Files
seedsDF = spark.read.csv("/FileStore/tables/WNCAATourneySeeds.csv", header=True, inferSchema= True)

slotsDF = spark.read.csv("/FileStore/tables/WNCAATourneySlots.csv", header=True, inferSchema=True)

#Read the TourneyResults CSV File
CompactDF = spark.read.csv("/FileStore/tables/WNCAATourneyCompactResults.csv", header=True, inferSchema= True)


#Read the Team details CSV file
teamsDF = spark.read.csv("/FileStore/tables/WTeams.csv", header=True, inferSchema= True)


In [4]:
#=nitial manipulation of data
wteamDF = wteamDF.toPandas()
lteamDF = lteamDF.toPandas()
regularSeasonsDF = regularSeasonsDF.toPandas()
slotsDF = slotsDF.toPandas()
seedsDF = seedsDF.toPandas()

#Rename the column to WTeamName
wteamDF.columns = ['WTeamID', 'WTeamName']
lteamDF.columns = ['LTeamID', 'LTeamName']

#Maitain a copy of the original data
NewseedsDF=seedsDF

CompactDF = CompactDF.toPandas()


In [5]:
#Create the DICTIONARY - team id,season as key and seed as value
seedsdict={}
for row in NewseedsDF.iterrows():
  seedsdict[(row[1]["Season"], row[1]["TeamID"])] = row[1]["Seed"]


In [6]:
#Merge the Season and team ID details from results DF and team DF 
temp_wCol = []
temp_lCol = []
for row in CompactDF.iterrows():
  year = row[1]['Season']
  wteamid = row[1]['WTeamID']
  lteamid = row[1]['LTeamID']
  temp_wCol.append(seedsdict[(year,wteamid)])
  temp_lCol.append(seedsdict[(year,lteamid)])
  

In [7]:
#Add the corresponding seed values into the dataframe
CompactDF['WSeed'] = temp_wCol
CompactDF['LSeed'] = temp_lCol

In [8]:
#Define weights for the seeds of each team - Meaning keep the highest weight of 16 for the team with Seed 1
weights_dict = {}
j = 1
for i in range(16,0,-1):
  weights_dict[j] = i
  j+=1


In [9]:
#PRE_PROCESSING THE DATAFRAME 

temp_win = []
wseed_num = []
lseed_num = []
diff_seed = []
loc_col = []
diff_score = []
low_team = []
high_team = []

for row in CompactDF.iterrows():
  
  team_1 = row[1]['WTeamID']
  team_2 = row[1]['LTeamID']
  loc_val = row[1]['WLoc']
  wseed = row[1]['WSeed']
  lseed =row[1]['LSeed']
  
  #Maintain the win column value as 1 if the team with lower teamID has won in the match
  if(team_1<team_2):
    temp_win.append(1)
    low_team.append(team_1)
    high_team.append(team_2)
  else:
    temp_win.append(0)
    high_team.append(team_1)
    low_team.append(team_2)
    
  #Give the highest weight when played in the home ground, least of outside home, medium vlaue otherwise   
  if(loc_val == 'H'):
    loc_col.append(3)
  elif(loc_val == 'N'):
    loc_col.append(2)
  elif(loc_val=='A'):
    loc_col.append(1)
    
  #Maintain the difference between seeds of the teams  
  temp_val = abs(weights_dict[int(wseed[1:])] - weights_dict[int(lseed[1:])])
  diff_seed.append(temp_val)
  
  #Maintain the column with difference between scores of the teams
  diff_score.append(abs(row[1]['WScore'] - row[1]['LScore']))
  

In [10]:
#Add the above obtained lists as columns into the DF
CompactDF['WLProb'] = temp_win
CompactDF['Seed_Diff'] = diff_seed
CompactDF['Loc'] = loc_col
CompactDF['Score_Diff'] = diff_score
CompactDF['Low_team'] = low_team
CompactDF['High_team'] = high_team

In [11]:
#Check the results of the pre-processing
CompactDF[:5]

In [12]:
#Conversion into the Spark SQL Dataframe
sqlCtx = SQLContext(sc)
sql_compactDF = sqlCtx.createDataFrame(CompactDF)

#Rename the result column with the name label so that all the algorithms can be applied without any problems
sql_compactDF= sql_compactDF.withColumnRenamed("WLProb", "label")

In [13]:
sql_compactDF.show()

In [14]:
#FEATURE ENGINEERING

#Definition of new features from existing data

#Obtain the totalMatches played and win percentage of the team in respective season

wDF = sql_compactDF.groupBy(['Season','WTeamID']).agg(fn.sum('label').alias('won'), fn.count('Season').alias('WCount'))
lDF = sql_compactDF.groupBy(['Season','LTeamID']).agg(fn.count('Season').alias('LCount'))

In [15]:
#Rename and maintain a clean DF
wDF = wDF.selectExpr("WTeamID as teamID", "Season", "won", "WCount")
lDF = lDF.selectExpr("LTeamID as teamID", "Season", "LCount")

In [16]:
#Create a DF of matches with the above combined details
matchDF = wDF.join(lDF, (wDF.teamID== lDF.teamID) & (wDF.Season==lDF.Season), how='right')

In [17]:
display(matchDF)

In [18]:
matchDF = matchDF.withColumn("totalMatches", sum([matchDF[col] for col in ['WCount', 'LCount']]))
#Computing the win percetage for the individaul teams
matchDF=matchDF.withColumn("winPercentage", fn.col('WCount')/fn.col('totalMatches') )
#Create Pandas DF only for this manipulation
#Renaming and selecting required data - avoiding redundancy
match_pd_DF = matchDF.toPandas()
match_pd_DF = match_pd_DF.iloc[:,[0,1,8]]
matchDF = sqlCtx.createDataFrame(match_pd_DF)

In [19]:
#Count and display the DF to ensure the join has not missed any data rows and other details
display(matchDF)
print(matchDF.count())
#Expect NaN because of the null values introduced during the join

In [20]:
#Add the details from matchDF to the initial integrated DF

winPercentage_DF = sql_compactDF.join(matchDF, (matchDF.teamID== sql_compactDF.WTeamID) & (sql_compactDF.Season==matchDF.Season), how='left').select('DayNum', sql_compactDF.Season, 'WTeamID', 'WScore', 'LTeamID', "LScore", 'NumOT', "WSeed", 'LSeed', 'label', 'Seed_Diff', 'Loc', "Score_Diff",fn.col('winPercentage').alias('W_win_percentage'))

winPercentage_DF = winPercentage_DF.join(matchDF, (matchDF.teamID== winPercentage_DF.LTeamID) & (winPercentage_DF.Season==matchDF.Season), how='left').select('DayNum', winPercentage_DF.Season, 'WTeamID', 'WScore', 'LTeamID', "LScore", 'NumOT', "WSeed", 'LSeed', 'label', 'Seed_Diff', 'Loc' ,"Score_Diff",'W_win_percentage',fn.col('winPercentage').alias('L_win_percentage'))

In [21]:
#Obtain the percentage as per the periods - 1998 to 2005; 2006 to 2010; 2011 to 2015; 2016 to 2017
#This further used as features in the models

groupedTeams_DF_1 = matchDF.where((fn.col('Season').cast(IntegerType())>=1998) & (fn.col('Season').cast(IntegerType())<=2005)).groupBy('teamID').agg(fn.avg('winPercentage').alias('1998_2005_win_percentage'))

#Repeat the grouping for remianing periods
groupedTeams_DF_2 = matchDF.where((fn.col('Season').cast(IntegerType())>=2006) & (fn.col('Season').cast(IntegerType())<=2010)).groupBy('teamID').agg(fn.avg('winPercentage').alias('2006_2010_win_percentage')) 
groupedTeams_DF_3 = matchDF.where((fn.col('Season').cast(IntegerType())>=2011) & (fn.col('Season').cast(IntegerType())<=2015)).groupBy('teamID').agg(fn.avg('winPercentage').alias('2011_2015_win_percentage'))
groupedTeams_DF_4 = matchDF.where((fn.col('Season').cast(IntegerType())>=2016) & (fn.col('Season').cast(IntegerType())<=2017)).groupBy('teamID').agg(fn.avg('winPercentage').alias('2016_2017_win_percentage')) 

In [22]:
#Removing null values 
winPercentage_DF=winPercentage_DF.na.fill(0)

#Display to check the final DF
display(winPercentage_DF)

In [23]:
#Join the data frame with the compact data frame
winPercentage_DF=groupedTeams_DF_1.join(winPercentage_DF, winPercentage_DF.WTeamID == groupedTeams_DF_1.teamID, how='right').select('DayNum', winPercentage_DF.Season, 'WTeamID', 'WScore', 'LTeamID', "LScore", 'NumOT', "WSeed", 'LSeed', 'label', 'Seed_Diff', 'Loc' ,"Score_Diff",'W_win_percentage', 'L_win_percentage', '1998_2005_win_percentage')

winPercentage_DF=groupedTeams_DF_2.join(winPercentage_DF, winPercentage_DF.WTeamID == groupedTeams_DF_2.teamID, how='right')
winPercentage_DF=groupedTeams_DF_3.join(winPercentage_DF, winPercentage_DF.WTeamID == groupedTeams_DF_3.teamID, how='right')
winPercentage_DF=groupedTeams_DF_4.join(winPercentage_DF, winPercentage_DF.WTeamID == groupedTeams_DF_4.teamID, how='right')


In [24]:
#Removing redundant columns
wpandasDF=winPercentage_DF.toPandas()
wpandasDF=wpandasDF.drop('teamID', axis=1)

#Creating spark sql dataframe
winPercentage_DF=sqlCtx.createDataFrame(wpandasDF)
#Removing null values
winPercentage_DF=winPercentage_DF.na.fill(0)



In [25]:
#Seed_Diff and Score_diff  - Histograms

winPercentage_DF.toPandas()[['Seed_Diff']].plot(kind='hist')
display()

In [26]:
winPercentage_DF.toPandas()[['Score_Diff']].plot(kind='hist')
display()

In [27]:
import seaborn as sns
#Categorical Variable - Loc versus Season
plt.figure()
wPandasDF = winPercentage_DF.toPandas()

ax = sns.boxplot(x=wPandasDF['Loc'], y=wPandasDF['Season'], hue=wPandasDF['Loc'], palette='Set3')
handles, _ = ax.get_legend_handles_labels()

ax.legend(handles, ['Away', 'Neutral','Home'])


display()

In [28]:
#Season versus W_win_percentage
plt.figure()
sns.stripplot(x='W_win_percentage', y='Season', data=wPandasDF[['W_win_percentage', 'Season', 'WTeamID']], jitter=0.5)

display()

In [29]:
plt.figure()
temp_win = wPandasDF[['label', '1998_2005_win_percentage','2006_2010_win_percentage', '2011_2015_win_percentage', '2016_2017_win_percentage', 'Season' ]]
sns.barplot(x='label',y='1998_2005_win_percentage', data=temp_win, hue='Season')
display()

In [30]:
#Barplot of 4 different 
plt.figure()
f, axes = plt.subplots(2, 2, figsize=(7, 7), sharex=True)
sns.despine(left=True)

sns.barplot(x='label',y='1998_2005_win_percentage', data=temp_win, ax=axes[0, 0])
sns.barplot(x='label',y='2006_2010_win_percentage', data=temp_win, ax=axes[0, 1])
sns.barplot(x='label',y='2011_2015_win_percentage', data=temp_win, ax=axes[1, 0])
sns.barplot(x='label',y='2016_2017_win_percentage', data=temp_win, ax=axes[1, 1])


display()

In [32]:
#INFERENCE OF THE ADDED FEATURES

#Checkimg the correlation between win percentages of each period with label
winPercentage_DF.select(fn.corr('1998_2005_win_percentage', 'label')).show()

In [33]:
winPercentage_DF.select(fn.corr('Loc', 'label')).show()

In [34]:
winPercentage_DF.select(fn.corr('2006_2010_win_percentage', 'label')).show()

In [35]:
winPercentage_DF.select(fn.corr('2011_2015_win_percentage', 'label')).show()

In [36]:
winPercentage_DF.select(fn.corr('Score_Diff', 'label')).show()

In [37]:
winPercentage_DF.select(fn.corr('Seed_Diff', 'label')).show()

In [38]:
winPercentage_DF.select(fn.corr('W_win_percentage', 'label')).show()

In [39]:
winPercentage_DF.select(fn.corr('L_win_percentage', 'label')).show()

In [40]:
winPercentage_DF.select(fn.corr('2016_2017_win_percentage', 'label')).show()

In [41]:
#Read the data with the team names integrated from the initial dataset
DF_with_teamnames = spark.read.csv('/FileStore/tables/wteamperc.csv',header=True, inferSchema= True)
display(DF_with_teamnames)

In [42]:
trainingData_4, validationData_4, testData_4 = DF_with_teamnames.randomSplit([0.6,0.3,0.1])

In [43]:
#####################################################MODEL 2 #########################################################
#Definition of the features with the win percentages of both the playing teams
featureCols_4 = ["WTeamID",'1998_2005_win_percentage', '2006_2010_win_percentage', '2011_2015_win_percentage', '2016_2017_win_percentage']

logistic_reg_4 = LogisticRegression()
assembler_4 = feature.VectorAssembler(inputCols=featureCols_4, outputCol="features")

pipeline_4 = Pipeline(stages=[ assembler_4, logistic_reg_4])

# Run stages in pipeline and train model
model_4 = pipeline_4.fit(trainingData_4)

In [44]:
val_predictions_4 = model_4.transform(validationData_4)

In [45]:
test_predictions_4 = model_4.transform(testData_4)

In [46]:
display(test_predictions_4)

In [47]:
#Construct the DF

predictions_DF = test_predictions_4.select('Season','WTeamName', 'LTeamName',fn.col('label').alias('Actual Result'), fn.col('prediction').alias('Predicted Result'))

In [48]:
display(predictions_DF)

In [49]:
teams_list_1 = predictions_DF.toPandas()['WTeamName']
teams_list_2 = predictions_DF.toPandas()['LTeamName']

In [50]:
teams_list = teams_list_1.append(teams_list_2)
len(teams_list)

In [51]:
#Remove any existing widgets from the current page
dbutils.widgets.removeAll()

In [52]:
#Create an interactive widget to take the input from the user for the team name so that the winning probability for the team can be predicted
dbutils.widgets.dropdown('TeamName','Missouri St', teams_list)

In [53]:
#Obtain the team name that was selected by the user in the widget
teamname=dbutils.widgets.get('TeamName')

In [54]:
#Register the predictions dataframe so that it can be queried to display the results
predictions_DF.registerTempTable('predictions_DF')

In [55]:
#Query the predictions data frame using the team name
x=predictions_DF.select('*').where((predictions_DF['WTeamName']==teamname) | (predictions_DF['LTeamName']==teamname))

In [56]:
#Show the results as per the user input
display(x)